# Newton's Law of Cooling - Parametric Study

This notebook demonstrates how to use the Funz-Modelica plugin to perform a parametric study of Newton's law of cooling.

## Problem Description

Newton's law of cooling states that the rate of heat loss of a body is proportional to the difference in temperature between the body and its surroundings:

$$\frac{dT}{dt} = -h \cdot A \cdot (T - T_{\infty})$$

Where:
- $T$ is the temperature of the object
- $T_{\infty}$ is the ambient temperature
- $h$ is the convective heat transfer coefficient
- $A$ is the surface area

We will study how different convection coefficients affect the cooling rate.

## 1. Create the Modelica Model

First, we define the Modelica model with a parametric variable `${convection~0.7}` that can be varied by Funz.

In [ ]:
# Create the Modelica model file
model_content = '''// @ref http://book.xogeny.com/behavior/equations/physical/
model NewtonCooling "An example of Newton's law of cooling"
  parameter Real T_inf=25 "Ambient temperature";
  parameter Real T0=90 "Initial temperature";
  parameter Real h=${convection~0.7} "Convective cooling coefficient";
  parameter Real A=1.0 "Surface area";
  parameter Real m=0.1 "Mass of thermal capacitance";
  parameter Real c_p=1.2 "Specific heat";
  Real T "Temperature";
initial equation
  T = T0 "Specify initial value for T";
equation
  m*c_p*der(T) = h*A*(T_inf-T) "Newton's law of cooling";
end NewtonCooling;
'''

# Write model to file
with open('NewtonCooling.mo', 'w') as f:
    f.write(model_content)

print("✓ Modelica model created: NewtonCooling.mo")
print(f"\nModel has parametric variable: convection (default=0.7)")

## 2. Run Single Simulation

Let's start by running a single simulation with the default convection coefficient.

In [ ]:
import fz
import pandas as pd

# Run a single case
print("Running Newton's cooling model with convection coefficient = 0.7")
results = fz.fzr(
    "NewtonCooling.mo",
    {"convection": 0.7},
    "Modelica",
    calculators="localhost",
    results_dir="results_single"
)

print("\n" + "="*60)
print("Results:")
print("="*60)
display(results)

# Extract and display temperature data
if len(results) > 0 and 'res' in results.columns:
    res_data = results.iloc[0]['res']['NewtonCooling']
    temps = list(res_data['T'].values())
    print(f"\nTemperature at t=0: {temps[0]:.2f}°C")
    print(f"Temperature at t=1: {temps[-1]:.2f}°C")

## 3. Parametric Study - Multiple Convection Coefficients

Now let's run a parametric study with different convection coefficients to see how they affect cooling.

In [ ]:
# Run parametric study
print("Running parametric study with multiple convection coefficients...")
convection_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.2, 1.5]

results_parametric = fz.fzr(
    "NewtonCooling.mo",
    {"convection": convection_values},
    "Modelica",
    calculators="localhost",
    results_dir="results_parametric"
)

print("\nResults summary:")
display(results_parametric[['convection', 'status']])

## 4. Visualize Results

Let's plot the temperature curves for different convection coefficients.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12, 7))

# Plot temperature vs time for each case
for idx, row in results_parametric.iterrows():
    if row['status'] == 'done' and 'res' in row:
        convection = row['convection']
        res_data = row['res']['NewtonCooling']
        time = list(res_data['time'].values())
        temp = list(res_data['T'].values())
        plt.plot(time, temp, marker='o', markersize=4, linewidth=2, 
                label=f'h={convection}', alpha=0.8)

plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Temperature (°C)', fontsize=12)
plt.title("Newton's Law of Cooling - Effect of Convection Coefficient", fontsize=14, fontweight='bold')
plt.legend(title='Convection Coefficient', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('cooling_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Plot saved as 'cooling_curves.png'")

## 5. Analysis - Cooling Rate

Let's analyze how the convection coefficient affects the final temperature reached.

In [ ]:
# Extract final temperatures
analysis_data = []
for idx, row in results_parametric.iterrows():
    if row['status'] == 'done' and 'res' in row:
        convection = row['convection']
        res_data = row['res']['NewtonCooling']
        temps = list(res_data['T'].values())
        final_temp = temps[-1]
        initial_temp = temps[0]
        temp_drop = initial_temp - final_temp
        
        analysis_data.append({
            'convection': convection,
            'initial_temp': initial_temp,
            'final_temp': final_temp,
            'temp_drop': temp_drop,
            'cooling_percent': (temp_drop / (initial_temp - 25)) * 100
        })

analysis_df = pd.DataFrame(analysis_data)
print("\nCooling Analysis:")
print("="*80)
display(analysis_df)

# Plot cooling efficiency
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Final temperature vs convection
ax1.plot(analysis_df['convection'], analysis_df['final_temp'], 'o-', linewidth=2, markersize=8)
ax1.axhline(y=25, color='r', linestyle='--', label='Ambient temp (25°C)')
ax1.set_xlabel('Convection Coefficient (h)', fontsize=11)
ax1.set_ylabel('Final Temperature (°C)', fontsize=11)
ax1.set_title('Final Temperature vs Convection', fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Cooling percentage vs convection
ax2.plot(analysis_df['convection'], analysis_df['cooling_percent'], 's-', 
         linewidth=2, markersize=8, color='green')
ax2.set_xlabel('Convection Coefficient (h)', fontsize=11)
ax2.set_ylabel('Cooling Progress (%)', fontsize=11)
ax2.set_title('Cooling Efficiency vs Convection', fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0, 105])

plt.tight_layout()
plt.savefig('cooling_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Analysis plot saved as 'cooling_analysis.png'")

## 6. Key Findings

From this parametric study, we observe:

1. **Higher convection coefficient = Faster cooling**: Objects cool more rapidly with higher h values
2. **Exponential approach**: All curves show exponential decay toward ambient temperature
3. **Asymptotic behavior**: Temperature approaches but never quite reaches ambient temperature in finite time
4. **Diminishing returns**: Very high convection coefficients show diminishing improvement

## Next Steps

Try modifying:
- Initial temperature (T0)
- Ambient temperature (T_inf)
- Mass and specific heat (m, c_p)
- Surface area (A)